In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

# An example of a Low Liquidity Trade

In [6]:
import requests
import s3fs
import pandas as pd
import re
import humps
import sqlalchemy as sa
from datetime import timedelta, datetime
import datetime as dt
import pendulum


import yfinance as yf

from prefect.client import Secret
from prefect import Client, Task, Flow, task, Parameter
from prefect.schedules import IntervalSchedule
from prefect.environments.storage import Docker

# Homemade
from extractMOCData import CONFIG as cfg
from extractMOCData.moc_data import TsxMocData

from normalize.ticker_symbols import MapTickerSymbols
from addFeatures.daily import DailyData 

from mocIO.read_moc import get_moc_file_lst
#from extractMOCData import read_moc 

import etl_moc

In [4]:
# tsxMocData = TsxMocData()

# moc_df = tsxMocData.scrape_moc_data()

## Load Raw Data

In [5]:
# See a list of tsx moc data
moc_file_lst = get_moc_file_lst.run("tsx-moc")
moc_file_lst

['tsx-moc/moc_20200324.csv',
 'tsx-moc/moc_20200325.csv',
 'tsx-moc/moc_20200326.csv',
 'tsx-moc/moc_20200327.csv',
 'tsx-moc/moc_20200330.csv',
 'tsx-moc/moc_tsx_20200331.csv',
 'tsx-moc/moc_tsx_20200401.csv',
 'tsx-moc/moc_tsx_20200402.csv',
 'tsx-moc/moc_tsx_20200403.csv',
 'tsx-moc/moc_tsx_20200406.csv',
 'tsx-moc/moc_tsx_20200407.csv',
 'tsx-moc/moc_tsx_20200409.csv',
 'tsx-moc/moc_tsx_20200413.csv',
 'tsx-moc/moc_tsx_20200414.csv',
 'tsx-moc/moc_tsx_20200415.csv',
 'tsx-moc/moc_tsx_20200416.csv',
 'tsx-moc/moc_tsx_20200417.csv',
 'tsx-moc/moc_tsx_20200420.csv',
 'tsx-moc/moc_tsx_20200421.csv']

In [ ]:
# Get all the files
data_flpth_lst = map(lambda flnm:  f"s3://{flnm}", moc_file_lst)
df_lst = [pd.read_csv(data_flpth, parse_dates=["moc_date"], na_filter=False) for data_flpth in list(data_flpth_lst)]
tsx_moc_df = pd.concat(df_lst , ignore_index=True)
tsx_moc_df.loc[tsx_moc_df["Symbol"]=="", "Symbol"] = "NA"

In [ ]:
tsx_moc_df[tsx_moc_df["Symbol"].isna()==True]

In [ ]:
tsx_moc_df["Symbol"].nunique()

## Create a key table
Map TSX symbols to Yahoo Symbols

In [ ]:
yhooMap =  MapTickerSymbols()
moc_key_df = yhooMap.run(tsx_moc_df)
moc_key_df["moc_date"] = moc_key_df["moc_date"].dt.date

In [ ]:
moc_key_df.info()

In [ ]:
moc_key_df["moc_date"] = moc_key_df["moc_date"].dt.date

## Get daily features
(from yahoo)

#### Price

In [ ]:
eod_price_df = etl_moc.get_eod_price_data.run(moc_key_df)

#### Other Attributes
(float, short interest)

In [ ]:
eod_info_df = etl_moc.get_sym_info.run(moc_key_df)

## Get indtraday features
(from yahoo)

In [ ]:
intraday_df = etl_moc.get_1min_ohlc.run(moc_key_df)

In [8]:
from prefect.client import Secret

s = Secret("moc_pgdb_conn") # create a secret object
conn_str = s.get()

In [18]:
# Read last date from tables
def get_date_sql(dStart, dEnd, engine):
    query = """ 
        SELECT * 
        FROM "intraday_prices" 
        WHERE "moc_date" >= %(start)s AND "moc_date" < %(end)s + interval '1 day'
    """
    query_params = {'start': dStart, 'end': dEnd}
    df1 = pd.read_sql_query(query, con=engine, params=query_params)
    return df1

intraday_df = get_date_sql(dt.date.today(), dt.date.today(), conn_str)

In [ ]:
eod_price_df = get_date_sql(dt.date.today(), dt.date.today(), conn_str)

In [17]:
eod_info_df

,moc_date,yahoo_symbol,tsx_symbol,zip,sector,long_business_summary,city,phone,state,country,...,category,five_year_average_return,regular_market_price,logo_url,full_time_employees,fax,underlying_symbol,underlying_exchange_symbol,head_symbol,uuid
0,2020-04-20,BB.TO,BB,N2K 0A7,Technology,BlackBerry Limited provides enterprise softwar...,Waterloo,519-888-7465,ON,Canada,...,None,None,5.91,https://logo.clearbit.com/blackberry.com,3647.0,519-888-6990,None,None,None,None
1,2020-04-20,BCE-PR.TO,BCE.PR.R,H3E 3B3,Communication Services,"BCE Inc., a telecommunications and media compa...",Verdun,800 339 6353,QC,Canada,...,None,None,11.38,https://logo.clearbit.com/bce.ca,52100.0,514 786 3970,None,None,None,17169013-9d2e-35a0-8a4a-528eb5b9fd23
2,2020-04-20,BCE-PT.TO,BCE.PR.T,H3E 3B3,Communication Services,"BCE Inc., a telecommunications and media compa...",Verdun,800 339 6353,QC,Canada,...,None,None,10.93,https://logo.clearbit.com/bce.ca,52100.0,514 786 3970,None,None,None,4c285cf1-fe98-3128-a4d5-75dae9aa4511
3,2020-04-20,BCE-PZ.TO,BCE.PR.Z,H3E 3B3,Communication Services,"BCE Inc., a telecommunications and media compa...",Verdun,800 339 6353,QC,Canada,...,None,None,11.96,https://logo.clearbit.com/bce.ca,52100.0,514 786 3970,None,None,None,082d1323-fca9-3a01-a303-d6dc9c172e23
4,2020-04-20,GRT-UN.TO,GRT.UN,M5K 1H1,Real Estate,Granite is a Canadian-based REIT engaged in th...,Toronto,647-925-7500,ON,Canada,...,None,None,60.12,https://logo.clearbit.com/granitereit.com,46.0,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,2020-04-20,WEF.TO,WEF,V6E 3P3,Basic Materials,Western Forest Products Inc. operates as an in...,Vancouver,604 648 4500,BC,Canada,...,None,None,0.65,https://logo.clearbit.com/westernforest.com,2210.0,604 681 9584,None,None,None,c421c6e0-e30d-399c-8c61-39fb55e5c495
589,2020-04-20,WFT.TO,WFT,V6B 1C1,Basic Materials,"West Fraser Timber Co. Ltd., a diversified woo...",Vancouver,604-895-2700,BC,Canada,...,None,None,29.54,https://logo.clearbit.com/westfraser.com,NaN,604-681-6061,None,None,None,a7ad8ab8-e258-336a-a92f-b72c0eab1963
590,2020-04-20,WIR-UN.TO,WIR.UN,M5L 1A9,,WPT Industrial Real Estate Investment Trust is...,Toronto,None,ON,Canada,...,None,None,14.87,https://logo.clearbit.com/wptreit.com,NaN,None,None,None,None,a9b22c29-36f4-3e8f-9496-4890e5d55df6
591,2020-04-20,WJX.TO,WJX,L5N 6A5,Industrials,Wajax Corporation operates an integrated distr...,Mississauga,905 813 8310,ON,Canada,...,None,None,7.15,https://logo.clearbit.com/wajax.com,NaN,905 812 7203,None,None,None,4e44d398-0aa1-354d-a0f7-8eba0f489072


In [15]:
eod_price_df

,moc_date,yahoo_symbol,tsx_symbol,imbalance_side,imbalance_size,imbalance_reference_price,adj_close,close,high,low,open,volume
0,2020-04-20,ABT.TO,ABT,BUY,200,9.975,10.03,10.03,10.06,9.73,10.04,37398.0
1,2020-04-20,ABX.TO,ABX,BUY,176352,35.305,35.24,35.24,35.79,34.20,34.27,4159584.0
2,2020-04-20,AC.TO,AC,BUY,20907,18.155,18.12,18.12,19.20,17.12,17.70,7190311.0
3,2020-04-20,ACB.TO,ACB,BUY,440883,0.995,1.02,1.02,1.04,0.95,0.97,7634469.0
4,2020-04-20,ACO-X.TO,ACO.X,BUY,19090,38.535,39.32,39.32,39.57,38.00,38.90,218317.0
...,...,...,...,...,...,...,...,...,...,...,...,...
588,2020-04-20,WEF.TO,WEF,SELL,2295,0.665,0.67,0.67,0.68,0.62,0.65,893702.0
589,2020-04-20,WFT.TO,WFT,SELL,4161,30.015,29.98,29.98,30.15,28.63,29.54,340870.0
590,2020-04-20,WIR-UN.TO,WIR.UN,SELL,10,15.120,10.94,10.94,10.94,10.31,10.71,85940.0
591,2020-04-20,WJX.TO,WJX,SELL,5505,7.635,7.60,7.60,7.70,6.96,7.15,119218.0


## Create MOC table

In [ ]:
moc_df = etl_moc.build_moc_data.run(intraday_df, eod_price_df, eod_info_df)

In [ ]:
moc_df

## Write to Db

In [ ]:
# 1. Eod
index_clmn_lst = ["moc_date", "yahoo_symbol"]

In [ ]:
num_rows_ins = etl_moc.df_to_db.run(intraday_df, tbl_name="intraday_prices", idx_clmn_lst=index_clmn_lst)

In [ ]:
# 2. Write to db
num_rows_ins = etl_moc.df_to_db.run(eod_price_df, tbl_name="eod_prices", idx_clmn_lst=index_clmn_lst)

# 3. Write to db
num_rows_ins = etl_moc.df_to_db.run(eod_info_df, tbl_name="eod_info", idx_clmn_lst=index_clmn_lst)

In [ ]:
# 4. Write to db
num_rows_ins = etl_moc.df_to_db.run(moc_df, tbl_name="daily_moc", idx_clmn_lst=index_clmn_lst)

In [ ]:
from prefect import task, Flow
from prefect.tasks.secrets import EnvVarSecret

@task
def print_value(x):
    print(x)

with Flow("Example") as flow:
    secret = EnvVarSecret("moc_pgdb_conn")
    print_value(secret)

In [ ]:
flow.run()

In [ ]:
import prefect
from prefect.client import Secret

In [ ]:
prefect.context.secrets["AWS_CREDENTIALS"] = 

In [ ]:
s = Secret("hello")

In [ ]:
s.exists()

In [ ]:
scrape_tsxmoc_fl.storage = Docker(dockerfile="/home/ilivni/MOC/Dockerfile")
scrape_tsxmoc_fl.register(project_name=project_nm)

In [ ]:
import boto3
session = boto3.session.Session()
client = session.client(
        service_name='secretsmanager'
    )
get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )

In [ ]:
secret = get_secret_value_response['SecretString']

In [ ]:
db_creds = get_db_creds()
type(db_creds)

In [ ]:
db_creds

In [ ]:
# Use this code snippet in your app.
# If you need more information about configurations or implementing the sample code, visit the AWS docs:
# https://aws.amazon.com/developers/getting-started/ruby/

# Use this code snippet in your app.
# If you need more information about configurations or implementing the sample code, visit the AWS docs:   
# https://aws.amazon.com/developers/getting-started/python/

import boto3
import base64
from botocore.exceptions import ClientError


def get_secret():

    secret_name = "moc-pg-db"
    region_name = "us-east-2"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            
    # Your code goes here. 

In [ ]:
Mschedule = IntervalSchedule(
    start_date=datetime.utcnow() + timedelta(seconds=1),
    interval=timedelta(minutes=1),
)
schedule

In [ ]:
moc_df["Symbol"] =moc_df["Symbol"].fillna("NA")

In [ ]:
moc_df[moc_df["Symbol"].isnull()]

In [ ]:
moc_df.to_json(orient="records")